In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import scvi

import sys
sys.path.extend(['../../scripts','../../scripts/xenium'])
import readwrite
cfg = readwrite.config()

/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/resolvi/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load data

In [8]:
# cfg paths
xenium_dir = Path(cfg['xenium_processed_data_dir'])
results_dir = Path(cfg['results_dir'])

# Segmentation, mapping paths
dir_segmentations = {
    dir_segmentation.name: (dir_segmentation)
    for dir_segmentation in xenium_dir.iterdir()
}

# Read resegmentations and RCTD
xenium_paths = {}
cc_paths = []

for segmentation in (segmentations := xenium_dir.iterdir()):
    for cohort in (cohorts := segmentation.iterdir()): 
        for panel in (panels := cohort.iterdir()):
            for sample in (samples := panel.iterdir()):
                for replicate in (replicates := sample.iterdir()):
                    
                    k = (segmentation.stem,cohort.stem,panel.stem,sample.stem,replicate.stem)
                    replicate_path = replicate / "normalised_results/outs"
                    name = '/'.join(k)

                    xenium_paths[k] = replicate_path

xenium_paths = {list(xenium_paths)[0]:xenium_paths[list(xenium_paths)[0]]}

xenium_levels = ('segmentation','cohort','panel','sample','replicate')
ads = readwrite.read_xenium_samples(xenium_paths,anndata_only=True,transcripts=False,sample_name_as_key=False)

Error processing The legacy implementation is no longer supported


In [14]:
ad = readwrite.read_xenium_sample('test',xenium_paths.values(),anndata_only=True,transcripts=False,)

NotImplementedError: The legacy implementation is no longer supported

In [9]:
ads

{}

In [4]:
ref_adata = ads[list(ads)[0]]
scvi.external.RESOLVI.setup_anndata(ref_adata, labels_key="predicted_celltype", layer="counts")
supervised_resolvi = scvi.external.RESOLVI(ref_adata, semisupervised=False)

IndexError: list index out of range